# 3.0 Interactive AI Agents for Information Retrieval

## **Introduction**

In this tutorial, we will:

- **Create an interactive AI agent** capable of searching for information based on user queries.
- **Leverage LangGraph** to manage the decision-making process, handling multiple tool calls and responses.
- **Visualize the workflow** of our AI agent to better understand its operations.


## **Workshop Outline**

1. **Setup**: Install libraries and prepare the environment.
2. **Agent Creation**: Design the agent and configure LangGraph.
3. **Tool Integration**: Implement search tools and handle responses.
4. **Workflow**: Manage the agent’s decision flow and responses.
5. **Visualization**: Visualize the decision-making process with LangGraph.
6. **Testing**: Interact with the agent through queries.
7. **Next Steps**: Enhance the agent and explore use cases.


By the end of this workshop, you’ll have hands-on experience in building interactive, intelligent agents capable of managing multiple tools and making decisions based on real-time queries!

### **Complete the Following Pre-Requisites**  
1. Select the kernel: **"Python(ai-agent)"**  
2. Perform **Clear all outputs**

### Step 1: Warning Control

In this cell, we are importing the `warnings` module and suppressing any warning messages that may appear during the execution of the code.  

- **`import warnings`**: This imports the Python `warnings` module, which is used to handle warning messages that are generated during code execution.
- **`warnings.filterwarnings('ignore')`**: This command tells Python to ignore all warnings. It's commonly used in notebooks to keep the output clean, especially when you know that certain warnings are not critical to the execution or the purpose of the notebook.

This step is helpful for reducing unnecessary clutter in the output, ensuring that only the important results are displayed.

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

### Step 2: Importing Libraries and Loading Environment Variables

We import essential libraries for building the AI agent:

- **`load_dotenv`**: Loads environment variables from a `.env` file.
- **`StateGraph` and `END` from LangGraph**: Manages state transitions and ends the process.
- **Type annotations**: `TypedDict` and `Annotated` help define structured data types.
- **Operator**: Used for basic operations like comparisons.
- **LangChain message types**: Handles different message types like `SystemMessage`, `HumanMessage`, etc.
- **`ChatOpenAI`**: Integrates GPT models for chat-based interactions.
- **`TavilySearchResults`**: Fetches search results from Tavily.

These libraries are key for managing the agent's interactions, state, and external searches.


In [ ]:
# Importing the required libraries
from dotenv import load_dotenv
_ = load_dotenv()

from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults


### Step 3: Initializing the Tools

In this step, we initialize the **TavilySearchResults** tool, which will allow our agent to retrieve information during its execution. 
This tool helps in fetching search results from Tavily, enabling the agent to gather relevant data for user queries.


In [ ]:
# Initialize the search tool with more results for a comprehensive answer
tool = TavilySearchResults(max_results=4) 
print(type(tool))  # Checking the tool type
print(tool.name)   # Displaying the tool's name

### Step 4: Defining the Agent State and Behavior

In this step, we define the agent's state and behavior. Using **LangGraph**, the agent will manage the flow between the decision-making process (handled by the language model, LLM) and the actions (such as retrieving information via the search tool). This ensures that the agent can intelligently process messages, make decisions, and perform actions accordingly.

In [ ]:
# Defining the agent's state with messages
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

# Define the Agent class
class Agent:
    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)  # Create a new state graph for the agent
        graph.add_node("llm", self.call_openai)  # Add LLM node for interacting with OpenAI
        graph.add_node("action", self.take_action)  # Add action node for interacting with tools
        graph.add_conditional_edges(
            "llm", 
            self.exists_action, 
            {True: "action", False: END}  # If action exists, move to 'action' node, else end
        )
        graph.add_edge("action", "llm")  # Return to LLM after action
        graph.set_entry_point("llm")  # Set entry point to LLM
        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}  # Map tool names to tool instances
        self.model = model.bind_tools(tools)  # Bind tools to the model

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0  # Check if tool calls are present

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages  # Add system message if provided
        message = self.model.invoke(messages)  # Invoke model to get a response
        return {'messages': [message]}  # Return response as a list of messages

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls  # Get tool calls from the last message
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:  # Check for valid tool name
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # Retry if tool name is invalid
            else:
                result = self.tools[t['name']].invoke(t['args'])  # Invoke the tool with arguments
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))  # Store the result
        print("Back to the model!")
        return {'messages': results}  # Return the results to the model


### Step 5: Defining the System Prompt and Initializing the Agent

In this step, we define a **system prompt** to guide the agent's behavior and initialize it with a language model and search tool.

In [ ]:
# Define the system prompt for the agent
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!"""

# Initialize the model and agent
model = ChatOpenAI(model="gpt-3.5-turbo")  # Using a lower cost model
abot = Agent(model, [tool], system=prompt)  # Initialize the agent with the model and tool


### Step 6: Visualizing the Agent's Decision Flow

In this step, we use **LangGraph** to visualize the agent's decision flow. This visualization helps us understand how the agent transitions between nodes, making decisions and taking actions based on user inputs and the available tools. By viewing the workflow, we gain insight into the agent's process, improving transparency and debugging capabilities.

In [ ]:
# Visualizing the agent's decision flow
import os
os.environ["PATH"] += os.pathsep + "/opt/homebrew/bin"
from IPython.display import Image
Image(abot.graph.get_graph().draw_png())  # Displaying the graph as a PNG image


### Step 7: Interacting with the Agent

In this step, we interact with the agent by providing user inputs. Based on the input, the agent will process the query and, if necessary, invoke the search tool to gather additional information. This allows the agent to dynamically respond to user queries and take actions accordingly.


In [ ]:
# Test the agent with a simple user input
messages = [HumanMessage(content="who won the latest nobel prize in physics?")]
result = abot.graph.invoke({"messages": messages})
print(result['messages'][-1].content)  # Output the result of the agent's response


In [ ]:
# Test with a more complex query
messages = [HumanMessage(content="Who won the 2024 nobel prize in physics? and when is his/her Birthday?")]
result = abot.graph.invoke({"messages": messages})
print(result['messages'][-1].content)  # Output the result of the agent's response

### Step 8: Advanced Query Handling

In this step, we test the agent with a more complex query. The agent will make multiple tool calls, processing each one sequentially, to gather the necessary information. After collecting the data, the agent will provide a more detailed and comprehensive answer based on the results.


In [ ]:
# Test with an advanced query
query = "Who won the Super Bowl in 2024? In what state is the winning team headquartered? \
What is the GDP of that state? Answer each question."
messages = [HumanMessage(content=query)]
model = ChatOpenAI(model="gpt-4o")  # Using a more advanced model for complex queries
abot = Agent(model, [tool], system=prompt)
result = abot.graph.invoke({"messages": messages})

# Output the result
print(result['messages'][-1].content)

### Step 9: Now let's run a UI for our Interactive AI Agent with Langgraph.

Go to the terminal and lets run this example in the browser for you to interact with it seemlessly.

1. open a new Terminal and then choose command prompt(cmd)
2. run this command >streamlit run C:\Users\admin\Desktop\devwks\DEVWKS-2382\UI\Interactice-AI-Agent-with-langgraph_UI.py

## Congratulations! You've built an interactive AI agent using LangGraph.

### Key Takeaways:
- **LangGraph** offers a structured framework for creating decision-making AI agents.
- **External tools**, like search engines, can be seamlessly integrated into the agent’s workflow for advanced capabilities.
- **Visualization** allows you to see and understand the agent's decision-making process, improving transparency.

Think about the many possibilities for using LangGraph in automating tasks, enhancing troubleshooting, or building even more intelligent and dynamic agents in the future!